# Intermediate Net in TF 

#### Load dependencies 

In [3]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

#### Load Data 

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


#### Set hyperparameters

In [5]:
lr = 0.1
epochs = 10
batch_size = 128
weight_initializer = tf.contrib.layers.xavier_initializer()

In [6]:
n_input = 784
n_dense_1 = 64
n_dense_2 = 64
n_classes = 10

#### Define placeholders 

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

#### Define layers 

In [8]:
def dense(x, W, b):
    z = tf.add(tf.matmul(x, W), b)
    a = tf.nn.relu(z)
    return a

#### Design neural network architecture 

In [9]:
def network(x, weights, biases):
    
    # two dense hidden layers:
    dense_1 = dense(x, weights['W1'], biases['b1'])
    dense_2 = dense(dense_1, weights['W2'], biases['b2'])    
    
    # linear output layer:
    out_layer_z = tf.add(tf.matmul(dense_2, weights['W_out']), biases['b_out'])
    
    return out_layer_z

#### Define variable dictionaries 

In [10]:
bias_dict = {
    'b1': tf.Variable(tf.zeros([n_dense_1])),
    'b2': tf.Variable(tf.zeros([n_dense_2])),
    'b_out': tf.Variable(tf.zeros([n_classes]))
}

weight_dict = {
    'W1': tf.get_variable('W1', [n_input, n_dense_1], initializer=weight_initializer),
    'W2': tf.get_variable('W2', [n_dense_1, n_dense_2], initializer=weight_initializer),
    'W_out': tf.get_variable('W_out', [n_dense_2, n_classes], initializer=weight_initializer)
}

#### Build Model 

In [11]:
predictions = network(x, weights=weight_dict, biases=bias_dict)

#### Define loss and optimizer 

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



#### Define evaluation metrics 

In [13]:
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy_pct = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))*100

#### Initialization Op 

In [14]:
initializer_op = tf.global_variables_initializer()

#### Train the network in session

In [18]:
with tf.Session() as session:
    session.run(initializer_op)
    
    print("Training for ", epochs, "epochs.")
    
    #loop over epochs:
    for epoch in range(epochs):
        
        avg_cost = 0.0
        avg_accuracy_pct = 0.0
        
        #loop over batches
        n_batches = int(mnist.train.num_examples / batch_size)
        for i in range(n_batches):
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            _, batch_cost, batch_acc = session.run([optimizer, cost, accuracy_pct],
                                                  feed_dict={x: batch_x, y: batch_y})
            
            avg_cost+= batch_cost / n_batches
            avg_accuracy_pct+= batch_acc / n_batches
        
        print("Epoch ", '%03d' % (epoch+1),
              ": cost =", '{:.3f}'.format(avg_cost),
              ", accuracy = ", '{:.2f}'.format(avg_accuracy_pct), "%",
                sep='')
        
    print("Training complete. Testing model.\n")

    test_cost = cost.eval({x: mnist.test.images, y: mnist.test.labels})
    test_accuracy_pct = accuracy_pct.eval({x: mnist.test.images, y: mnist.test.labels}) 

    print("Test Cost:", '{:.3f}'.format(test_cost))
    print("Test Accuracy: ", '{:.2f}'.format(test_accuracy_pct), "%", sep='')

Training for  10 epochs.
Epoch 001: cost =0.492, accuracy = 85.82%
Epoch 002: cost =0.246, accuracy = 92.77%
Epoch 003: cost =0.192, accuracy = 94.30%
Epoch 004: cost =0.159, accuracy = 95.33%
Epoch 005: cost =0.137, accuracy = 95.88%
Epoch 006: cost =0.119, accuracy = 96.55%
Epoch 007: cost =0.107, accuracy = 96.86%
Epoch 008: cost =0.096, accuracy = 97.18%
Epoch 009: cost =0.087, accuracy = 97.47%
Epoch 010: cost =0.079, accuracy = 97.67%
Training complete. Testing model.

Test Cost: 0.098
Test Accuracy: 97.14%
